### Only for Dev

If you are debugging this library or developing some new features, please don't imports as in other tutorials as they can load the pyvene library installed in your local env without your code updates.

**Note**: If is better to remove your local pyvene pip install before dev.

**Never Imports Like This**

In [ ]:
# try:
#     # This library is our indicator that the required installs
#     # need to be done.
#     import pyvene

# except ModuleNotFoundError:
#     !pip install git+https://github.com/frankaging/pyvene.git

**Do Relative Imports Instead**

This way your updated code will be loaded instead of using your installed pyvene library.

In [ ]:
import sys
sys.path.append("../..")

from pyvene.models.basic_utils import (
    embed_to_distrib,
    top_vals,
    format_token,
    count_parameters
)

from pyvene.models.gpt2.modelings_intervenable_gpt2 import create_gpt2

from pyvene.models.intervenable_base import IntervenableModel
from pyvene.models.interventions import VanillaIntervention

from pyvene.models.configuration_intervenable_model import (
    IntervenableConfig, IntervenableRepresentationConfig, VanillaIntervention)

config, tokenizer, gpt = create_gpt2(cache_dir="../../../.huggingface_cache")

**tensor versioning**

In [ ]:
import torch

when a tensor is created, its `_version` is 0

In [ ]:
a = torch.rand(3,3)
b = torch.rand(3,3)
w = torch.rand(3,3)
a._version

if there is any in-place op on this tensor, the `_version` number will increment by 1 for each op

In [18]:
a[0,0] = 0
b[0,0] = 0
a._version

1

for op that does not have side-effect (i.e., directly return), it creats new tensors, thus the `_version` for the output is 0 again, since it is a new tensor.

In [19]:
(w @ a)._version

0

In [20]:
(b @ a)._version

0

In [22]:
a.clone()._version

0

**hook fail safe**

if your hook ends up causing downstream calculation error, pytorch will actually absorb it and remove the hook. so, you need to be careful!

In [3]:
import torch
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(5, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

net = SimpleNet()
    
def output_str(self, input, output):
    output = "hey!"

# Step 3: Attach the Hook to a Layer
hook = net.fc1.register_forward_hook(output_str)

# Step 4: Run a Forward Pass
input = torch.randn(1, 10)
output = net(input)

# Remove the hook if it's no longer needed
hook.remove()
output

tensor([[-0.1888,  0.3058]], grad_fn=<AddmmBackward0>)